# Pandas TA ([pandas_ta](https://github.com/twopirllc/pandas-ta)) Strategies for Custom Technical Analysis

## Topics
- What is a Pandas TA Strategy?
    - Builtin Strategies: __AllStrategy__ and __CommonStrategy__
    - Creating Strategies
- Watchlist Class
    - Strategy Management and Execution
- Indicator Composition/Chaining for more Complex Strategies
    - Comprehensive Example: _MACD and RSI Momo with BBANDS and SMAs 50 & 200 and Cumulative Log Returns_

In [1]:
%matplotlib inline
import datetime as dt

# import matplotlib.pyplot as plt
# import mplfinance as mpf

import pandas as pd
import pandas_ta as ta
from alphaVantageAPI.alphavantage import AlphaVantage  # pip install alphaVantage-api

from watchlist import Watchlist
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# What is a Pandas TA Strategy?
A _Strategy_ is a simple way to name and group your favorite TA indicators. Technically, a _Strategy_ is a simple Data Class to contain list of indicators and their parameters. __Note__: _Strategy_ is experimental and subject to change. Pandas TA comes with two basic Strategies: __AllStrategy__ and __CommonStrategy__.

## Strategy Requirements:
- _name_: Some short memorable string.  _Note_: Case-insensitive "All" is reserved.
- _ta_: A list of dicts containing keyword arguments to identify the indicator and the indicator's arguments

## Optional Requirements:
- _description_: A more detailed description of what the Strategy tries to capture. Default: None
- _created_: At datetime string of when it was created. Default: Automatically generated.

### Things to note:
- A Strategy will __fail__ when consumed by Pandas TA if there is no {"kind": "indicator name"} attribute.

# Builtin Examples

### All

In [2]:
AllStrategy = ta.AllStrategy
print("name =", AllStrategy.name)
print("description =", AllStrategy.description)
print("created =", AllStrategy.created)
print("ta =", AllStrategy.ta)

name = All
description = All the indicators with their default settings. Pandas TA default.
created = 07/25/2020, 11:24:19
ta = None


### Common

In [3]:
CommonStrategy = ta.CommonStrategy
print("name =", CommonStrategy.name)
print("description =", CommonStrategy.description)
print("created =", CommonStrategy.created)
print("ta =", CommonStrategy.ta)

name = Common Price and Volume SMAs
description = Common Price SMAs: 10, 20, 50, 200 and Volume SMA: 20.
created = 07/25/2020, 11:24:19
ta = [{'kind': 'sma', 'length': 10}, {'kind': 'sma', 'length': 20}, {'kind': 'sma', 'length': 50}, {'kind': 'sma', 'length': 200}, {'kind': 'sma', 'close': 'volume', 'length': 20, 'prefix': 'VOL'}]


# Creating Strategies

### Simple Strategy A

In [4]:
custom_a = ta.Strategy(name="A", ta=[{"kind": "sma", "length": 50}, {"kind": "sma", "length": 200}])
custom_a

Strategy(name='A', ta=[{'kind': 'sma', 'length': 50}, {'kind': 'sma', 'length': 200}], description=None, created='07/25/2020, 11:24:19', last_run=None, run_time=None)

### Simple Strategy B

In [5]:
custom_b = ta.Strategy(name="B", ta=[{"kind": "ema", "length": 8}, {"kind": "ema", "length": 21}, {"kind": "log_return", "cumulative": True}, {"kind": "rsi"}, {"kind": "supertrend"}])
custom_b

Strategy(name='B', ta=[{'kind': 'ema', 'length': 8}, {'kind': 'ema', 'length': 21}, {'kind': 'log_return', 'cumulative': True}, {'kind': 'rsi'}, {'kind': 'supertrend'}], description=None, created='07/25/2020, 11:24:19', last_run=None, run_time=None)

### Bad Strategy. (Misspelled Indicator)

In [6]:
# Misspelled indicator, will fail later when ran with Pandas
custom_run_failure = ta.Strategy(name="Runtime Failure", ta=[{"kind": "percet_return"}])
custom_run_failure

Strategy(name='Runtime Failure', ta=[{'kind': 'percet_return'}], description=None, created='07/25/2020, 11:24:19', last_run=None, run_time=None)

# Strategy Management and Execution with _Watchlist_

### Initialize AlphaVantage Data Source

In [7]:
AV = AlphaVantage(
    api_key="YOUR API KEY", premium=False,
    output_size='full', clean=True,
    export_path=".", export=True
)
AV

AlphaVantage(
  end_point:str = https://www.alphavantage.co/query,
  api_key:str = YOUR API KEY,
  export:bool = True,
  export_path:str = .,
  output_size:str = full,
  output:str = csv,
  datatype:str = json,
  clean:bool = True,
  proxy:dict = {}
)

### Create Watchlist and set it's 'ds' to AlphaVantage

In [8]:
watch = Watchlist(["SPY", "IWM"], ds=AV)

#### Info about the Watchlist. Note, the default Strategy is "All"

In [9]:
watch

Watch(name='Watchlist: SPY, IWM', tickers[2]='SPY, IWM', tf='D', strategy[0]='All')

### Help about Watchlist

In [10]:
help(Watchlist)

Help on class Watchlist in module watchlist:

class Watchlist(builtins.object)
 |  Watchlist(tickers: list, tf: str = None, name: str = None, strategy: pandas_ta.core.Strategy = None, ds: object = None, **kwargs)
 |  
 |  Watchlist Class (** This is subject to change! **)
 |  ============================================================================
 |  A simple Class to load/download financial market data and automatically
 |  apply Technical Analysis indicators with a Pandas TA Strategy. Default
 |  Strategy: pandas_ta.AllStrategy.
 |  
 |  Requirements:
 |  - Pandas TA (pip install pandas_ta)
 |  - AlphaVantage (pip install alphaVantage-api) for the Default Data Source.
 |      To use another Data Source, update the load() method after AV.
 |  
 |  Required Arguments:
 |  - tickers: A list of strings containing tickers. Example: ['SPY', 'AAPL']
 |  ============================================================================
 |  
 |  Methods defined here:
 |  
 |  __init__(self, ti

### Default Strategy is "All"

In [11]:
# No arguments loads all the tickers and applies the Strategy to each ticker.
# The result can be accessed with Watchlist's 'data' property which returns a dictionary keyed by ticker and DataFrames as values 
watch.load(verbose=True, timed=False)

[!] Loading All: SPY, IWM

[+] Downloading['D']: SPY
[+] Strategy: All
[i] Indicators with the following arguments: {'append': True}
[i] Excluded[10]: above, above_value, below, below_value, cross, cross_value, long_run, short_run, trend_return, vp
[i] Total indicators: 101
[i] Columns added: 152

[+] Downloading['D']: IWM
[+] Strategy: All
[i] Indicators with the following arguments: {'append': True}
[i] Excluded[10]: above, above_value, below, below_value, cross, cross_value, long_run, short_run, trend_return, vp
[i] Total indicators: 101
[i] Columns added: 152


In [12]:
watch.data

{'SPY':                 open      high       low     close      volume  ABER_ZG_5_15  \
 date                                                                           
 1999-11-01  136.5000  137.0000  135.5625  135.5625   4006500.0           NaN   
 1999-11-02  135.9687  137.2500  134.5937  134.5937   6516900.0           NaN   
 1999-11-03  136.0000  136.3750  135.1250  135.5000   7222300.0           NaN   
 1999-11-04  136.7500  137.3593  135.7656  136.5312   7907500.0           NaN   
 1999-11-05  138.6250  139.1093  136.7812  137.8750   7431500.0    136.332267   
 ...              ...       ...       ...       ...         ...           ...   
 2020-07-20  321.4300  325.1300  320.6200  324.3200  56150230.0    320.673333   
 2020-07-21  326.4500  326.9300  323.9400  325.0100  57245315.0    322.353333   
 2020-07-22  324.6200  327.2000  324.5000  326.8600  57792915.0    323.313333   
 2020-07-23  326.4700  327.2300  321.4800  322.9600  75737989.0    324.014000   
 2020-07-24  320.9500

### 

In [13]:
watch.data['SPY']

,open,high,low,close,volume,ABER_ZG_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ATR_5_15,ACCBL_20,...,VAR_30,VTXP_14,VTXM_14,VWAP,VWMA_10,WCP,WILLR_14,WMA_10,ZL_EMA_10,Z_30
date,,,,,,,,,,,,,,,,,,,,,
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,136.041667,NaN,135.921875,NaN,NaN,NaN,NaN
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,135.693303,NaN,135.257775,NaN,NaN,NaN,NaN
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,135.682462,NaN,135.625000,NaN,NaN,NaN,NaN
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,135.950504,NaN,136.546825,NaN,NaN,NaN,NaN
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500.0,136.332267,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,136.393305,NaN,137.910125,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-20,321.4300,325.1300,320.6200,324.3200,56150230.0,320.673333,326.296123,315.050544,5.622790,300.906776,...,45.091757,1.223611,0.623952,153.261128,318.474591,323.597500,-3.801032,320.096545,323.215081,1.680556
2020-07-21,326.4500,326.9300,323.9400,325.0100,57245315.0,322.353333,327.800604,316.906063,5.447270,301.895656,...,46.469833,1.211493,0.661493,153.278065,319.600245,325.222500,-10.750280,321.291636,324.115975,1.747819
2020-07-22,324.6200,327.2000,324.5000,326.8600,57792915.0,323.313333,328.577452,318.049214,5.264119,302.558926,...,50.919143,1.153997,0.689436,153.295250,320.677611,326.355000,-2.058111,322.618909,325.718525,1.900614


## Easy to swap Strategies and run them

### Running Simple Strategy A

In [14]:
# Load custom_a into Watchlist and verify
watch.strategy = custom_a
watch.strategy

Strategy(name='A', ta=[{'kind': 'sma', 'length': 50}, {'kind': 'sma', 'length': 200}], description=None, created='07/25/2020, 11:24:19', last_run=None, run_time=None)

In [15]:
watch.load('IWM')


[i] Loaded['D']: IWM_D.csv


,open,high,low,close,volume,SMA_50,SMA_200
date,,,,,,,
2000-05-26,91.06,91.440,90.63,91.44,37400.0,NaN,NaN
2000-05-30,92.75,94.810,92.75,94.81,28800.0,NaN,NaN
2000-05-31,95.13,96.380,95.13,95.75,18000.0,NaN,NaN
2000-06-01,97.11,97.310,97.11,97.31,3500.0,NaN,NaN
2000-06-02,101.70,102.400,101.70,102.40,14700.0,NaN,NaN
...,...,...,...,...,...,...,...
2020-07-20,146.12,146.850,145.15,145.96,19581689.0,139.7062,145.93605
2020-07-21,147.47,149.160,147.20,148.03,24467065.0,140.0196,145.93750
2020-07-22,147.09,148.670,147.03,148.11,24424808.0,140.3478,145.93235


### Running Simple Strategy B

In [16]:
# Load custom_b into Watchlist and verify
watch.strategy = custom_b
watch.strategy

Strategy(name='B', ta=[{'kind': 'ema', 'length': 8}, {'kind': 'ema', 'length': 21}, {'kind': 'log_return', 'cumulative': True}, {'kind': 'rsi'}, {'kind': 'supertrend'}], description=None, created='07/25/2020, 11:24:19', last_run=None, run_time=None)

In [17]:
watch.load('SPY')


[i] Loaded['D']: SPY_D.csv


,open,high,low,close,volume,EMA_8,EMA_21,CUMLOGRET_1,RSI_14,SUPERT_7_3.0,SUPERTd_7_3.0,SUPERTl_7_3.0,SUPERTs_7_3.0
date,,,,,,,,,,,,,
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500.0,NaN,NaN,NaN,NaN,0.000000,1,NaN,NaN
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900.0,NaN,NaN,-0.007172,0.000000,NaN,1,NaN,NaN
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300.0,NaN,NaN,-0.000461,50.185503,NaN,1,NaN,NaN
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500.0,NaN,NaN,0.007120,69.153995,NaN,1,NaN,NaN
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500.0,NaN,NaN,0.016915,79.896816,NaN,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-20,321.4300,325.1300,320.6200,324.3200,56150230.0,319.783320,315.076450,0.872298,63.569724,307.890439,1,307.890439,NaN
2020-07-21,326.4500,326.9300,323.9400,325.0100,57245315.0,320.944805,315.979500,0.874423,64.179426,311.309662,1,311.309662,NaN
2020-07-22,324.6200,327.2000,324.5000,326.8600,57792915.0,322.259293,316.968636,0.880099,65.830627,312.585425,1,312.585425,NaN


### Running Bad Strategy. (Misspelled indicator)

In [18]:
# Load custom_run_failure into Watchlist and verify
watch.strategy = custom_run_failure
watch.strategy

Strategy(name='Runtime Failure', ta=[{'kind': 'percet_return'}], description=None, created='07/25/2020, 11:24:19', last_run=None, run_time=None)

In [19]:
try:
    iwm = watch.load('IWM')
except AttributeError as error:
    print(f"[X] Oops! {error}")


[i] Loaded['D']: IWM_D.csv
[X] Oops! 'AnalysisIndicators' object has no attribute 'percet_return'


# Indicator Composition/Chaining
- When you need an indicator to depend on the value of a prior indicator
- Utilitze _prefix_ or _suffix_ to help identify unique columns or avoid column name clashes.

### Volume MAs and MA chains

In [20]:
# Set EMA's and SMA's 'close' to 'volume' to create Volume MAs, prefix 'volume' MAs with 'VOLUME' so easy to identify the column
# Take a price EMA and apply LINREG from EMA's output
volmas_price_ma_chain = [
    {"kind":"ema", "close": "volume", "length": 10, "prefix": "VOLUME"},
    {"kind":"sma", "close": "volume", "length": 20, "prefix": "VOLUME"},
    {"kind":"ema", "length": 5},
    {"kind":"linreg", "close": "EMA_5", "length": 8, "prefix": "EMA_5"},
]
vp_ma_chain_ta = ta.Strategy("Volume MAs and Price MA chain", volmas_price_ma_chain)
vp_ma_chain_ta

Strategy(name='Volume MAs and Price MA chain', ta=[{'kind': 'ema', 'close': 'volume', 'length': 10, 'prefix': 'VOLUME'}, {'kind': 'sma', 'close': 'volume', 'length': 20, 'prefix': 'VOLUME'}, {'kind': 'ema', 'length': 5}, {'kind': 'linreg', 'close': 'EMA_5', 'length': 8, 'prefix': 'EMA_5'}], description=None, created='07/25/2020, 11:24:19', last_run=None, run_time=None)

In [21]:
# Update the Watchlist
watch.strategy = vp_ma_chain_ta
watch.strategy.name

'Volume MAs and Price MA chain'

In [22]:
spy = watch.load('SPY')
spy


[i] Loaded['D']: SPY_D.csv


,open,high,low,close,volume,VOLUME_EMA_10,VOLUME_SMA_20,EMA_5,EMA_5_LR_8
date,,,,,,,,,
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500.0,NaN,NaN,NaN,NaN
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900.0,NaN,NaN,NaN,NaN
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300.0,NaN,NaN,NaN,NaN
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500.0,NaN,NaN,NaN,NaN
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500.0,NaN,NaN,136.012480,NaN
...,...,...,...,...,...,...,...,...,...
2020-07-20,321.4300,325.1300,320.6200,324.3200,56150230.0,7.254859e+07,81019145.80,321.398416,320.058000
2020-07-21,326.4500,326.9300,323.9400,325.0100,57245315.0,6.976618e+07,80181050.95,322.602277,321.307000
2020-07-22,324.6200,327.2000,324.5000,326.8600,57792915.0,6.758922e+07,79667351.70,324.021518,322.687127


### MACD BBANDS

In [23]:
# MACD is the initial indicator that BBANDS depends on.
# Set BBANDS's 'close' to MACD's main signal, in this case 'MACD_12_26_9' and add a prefix (or suffix) so it's easier to identify
macd_bands_ta = [
    {"kind":"macd"},
    {"kind":"bbands", "close": "MACD_12_26_9", "length": 20, "prefix": "MACD"}
]
macd_bands_ta = ta.Strategy("MACD BBands", macd_bands_ta, f"BBANDS_{macd_bands_ta[1]['length']} applied to MACD")
macd_bands_ta

Strategy(name='MACD BBands', ta=[{'kind': 'macd'}, {'kind': 'bbands', 'close': 'MACD_12_26_9', 'length': 20, 'prefix': 'MACD'}], description='BBANDS_20 applied to MACD', created='07/25/2020, 11:24:19', last_run=None, run_time=None)

In [24]:
# Update the Watchlist
watch.strategy = macd_bands_ta
watch.strategy.name

'MACD BBands'

In [25]:
spy = watch.load('SPY')
spy


[i] Loaded['D']: SPY_D.csv
[i] Set 'df.ta.mp = True' to enable multiprocessing. This computer has 4 cores. Default: False


,open,high,low,close,volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,MACD_BBL_20_2.0,MACD_BBM_20_2.0,MACD_BBU_20_2.0
date,,,,,,,,,,,
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500.0,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900.0,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300.0,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500.0,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-07-20,321.4300,325.1300,320.6200,324.3200,56150230.0,4.422827,0.713695,3.709131,1.351168,3.153296,4.955423
2020-07-21,326.4500,326.9300,323.9400,325.0100,57245315.0,4.651974,0.754274,3.897700,1.341550,3.157320,4.973090
2020-07-22,324.6200,327.2000,324.5000,326.8600,57792915.0,4.926070,0.822696,4.103374,1.279092,3.183563,5.088035


# Comprehensive Strategy

### MACD and RSI Momentum with BBANDS and SMAs and Cumulative Log Returns

In [26]:
momo_bands_sma_ta = [
    {"kind":"sma", "length": 50},
    {"kind":"sma", "length": 200},
    {"kind":"bbands", "length": 20},
    {"kind":"macd"},
    {"kind":"rsi"},
    {"kind":"log_return", "cumulative": True},
    {"kind":"sma", "close": "CUMLOGRET_1", "length": 5, "suffix": "CUMLOGRET"},
]
momo_bands_sma_strategy = ta.Strategy(
    "Momo, Bands and SMAs and Cumulative Log Returns", # name
    momo_bands_sma_ta, # ta
    "MACD and RSI Momo with BBANDS and SMAs 50 & 200 and Cumulative Log Returns" # description
)
momo_bands_sma_strategy

Strategy(name='Momo, Bands and SMAs and Cumulative Log Returns', ta=[{'kind': 'sma', 'length': 50}, {'kind': 'sma', 'length': 200}, {'kind': 'bbands', 'length': 20}, {'kind': 'macd'}, {'kind': 'rsi'}, {'kind': 'log_return', 'cumulative': True}, {'kind': 'sma', 'close': 'CUMLOGRET_1', 'length': 5, 'suffix': 'CUMLOGRET'}], description='MACD and RSI Momo with BBANDS and SMAs 50 & 200 and Cumulative Log Returns', created='07/25/2020, 11:24:19', last_run=None, run_time=None)

In [27]:
# Update the Watchlist
watch.strategy = momo_bands_sma_strategy
watch.strategy.name

'Momo, Bands and SMAs and Cumulative Log Returns'

In [28]:
spy = watch.load('SPY', timed=True)
# Apply constants to the DataFrame for indicators
spy.ta.constants(True, 0, 0, 1) # 0
spy.ta.constants(True, 30, 30, 1) # 30
spy.ta.constants(True, 70, 70, 1) # 70
spy.head(20)


[i] Loaded['D']: SPY_D.csv
[i] Runtime: 34.4336 ms (0.0344 s)


,open,high,low,close,volume,SMA_50,SMA_200,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,CUMLOGRET_1,SMA_5_CUMLOGRET,0,30,70
date,,,,,,,,,,,,,,,,,,,
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,30,70
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.007172,NaN,0,30,70
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.185503,-0.000461,NaN,0,30,70
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.153995,0.007120,NaN,0,30,70
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.896816,0.016915,NaN,0,30,70
1999-11-08,137.0000,138.3750,136.7500,138.0000,4649200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.574537,0.017821,0.006845,0,30,70
1999-11-09,138.5000,138.6875,136.2812,136.7031,4533700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.528352,0.008379,0.009955,0,30,70
1999-11-10,136.2500,138.3906,136.0781,137.7187,6405600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.303684,0.015780,0.013203,0,30,70
1999-11-11,138.1875,138.5000,137.4687,138.5000,4794100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,70.833962,0.021438,0.016066,0,30,70
